l

In [1]:
!wget https://archive.ics.uci.edu/static/public/275/bike+sharing+dataset.zip

--2024-08-27 18:07:17--  https://archive.ics.uci.edu/static/public/275/bike+sharing+dataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘bike+sharing+dataset.zip.1’

bike+sharing+datase     [ <=>                ] 273.43K  --.-KB/s    in 0.05s   

2024-08-27 18:07:18 (5.12 MB/s) - ‘bike+sharing+dataset.zip.1’ saved [279992]



In [2]:
!unzip bike+sharing+dataset.zip

Archive:  bike+sharing+dataset.zip
replace Readme.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Readme.txt              
replace day.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: day.csv                 
replace hour.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: hour.csv                


In [3]:
# @title
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [4]:
df = pd.read_csv('hour.csv')


In [5]:
# Feature engineering
df['day_night'] = df['hr'].apply(lambda x: 'day' if 6 <= x <= 18 else 'night')
df.drop(['instant', 'casual', 'registered'], axis=1, inplace=True)
df['dteday'] = pd.to_datetime(df.dteday)
df['season'] = df.season.astype('category')
df['holiday'] = df.holiday.astype('category')
df['weekday'] = df.weekday.astype('category')
df['weathersit'] = df.weathersit.astype('category')
df['workingday'] = df.workingday.astype('category')
df['mnth'] = df.mnth.astype('category')
df['yr'] = df.yr.astype('category')
df['hr'] = df.hr.astype('category')
df.drop(columns=['dteday'], inplace=True)

In [6]:
# Creating interaction features
df['temp_hum'] = df['temp'] * df['hum']
df['temp_windspeed'] = df['temp'] * df['windspeed']

temp_hum: factors comfort for people, as depending on combination of temperature and humidity the envirnment can be favourablke or unfavourable for people to go cycling. This interaction allows the model to account for scenarios where the joint impact of these variables influences user behavior better than their individual effects would allow for

temp_windspeed: wndspeeds can decrease precieved temperatures, which can encourage more people to rent and go cycling. It can improve feature richness as well and account for peoples behaviour depending on weather conditions

In [7]:
# Separating features and target variable
X = df.drop(columns=['cnt']) # Features
y = df['cnt'] # Target

In [8]:
X.columns = X.columns.astype(str)

In [9]:
!pip install category_encoders

y


In [10]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from category_encoders import TargetEncoder

# Numerical features
numerical_features = ['temp', 'hum', 'windspeed', 'temp_hum', 'temp_windspeed']
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values
    ('scaler', MinMaxScaler())  # Normalize
])
X[numerical_features] = numerical_pipeline.fit_transform(X[numerical_features])

# Categorical features
categorical_features = ['season', 'weathersit', 'day_night']
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values
    ('target_encode', TargetEncoder())  # Target encoding
])
X[categorical_features] = categorical_pipeline.fit_transform(X[categorical_features], y)
'''
X_encoded = categorical_pipeline.fit_transform(X[categorical_features], y)

# Combine all features
X = pd.concat([X.drop(columns=categorical_features), X_encoded], axis=1)
'''
# Convert all column names to strings to avoid type issues
X.columns = X.columns.astype(str)


In [11]:
print(X.head())

       season yr mnth hr holiday weekday workingday  weathersit      temp  \
0  111.114569  0    1  0       0       6          0  204.869272  0.224490   
1  111.114569  0    1  1       0       6          0  204.869272  0.204082   
2  111.114569  0    1  2       0       6          0  204.869272  0.204082   
3  111.114569  0    1  3       0       6          0  204.869272  0.224490   
4  111.114569  0    1  4       0       6          0  204.869272  0.224490   

    atemp   hum  windspeed  day_night  temp_hum  temp_windspeed  
0  0.2879  0.81        0.0  98.894138  0.312039             0.0  
1  0.2727  0.80        0.0  98.894138  0.282504             0.0  
2  0.2727  0.80        0.0  98.894138  0.282504             0.0  
3  0.2879  0.75        0.0  98.894138  0.288925             0.0  
4  0.2879  0.75        0.0  98.894138  0.288925             0.0  


In [12]:
# Check for NaN values in the entire dataset
print(X.isnull().sum())  # Verify there are no NaNs

season            0
yr                0
mnth              0
hr                0
holiday           0
weekday           0
workingday        0
weathersit        0
temp              0
atemp             0
hum               0
windspeed         0
day_night         0
temp_hum          0
temp_windspeed    0
dtype: int64


In [13]:
# Convert all column names to strings to avoid type issues
X.columns = X.columns.astype(str)

In [14]:
from sklearn.model_selection import train_test_split
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [15]:
'''# @title
feat_importances = pd.DataFrame(model.feature_importances_, index=X_train.columns,
columns=["Importance"])
feat_importances.sort_values(by='Importance', ascending=False, inplace=True)
# Plotting each feature importance
feat_importances.plot(kind='bar', figsize=(12,8))'''

'# @title\nfeat_importances = pd.DataFrame(model.feature_importances_, index=X_train.columns,\ncolumns=["Importance"])\nfeat_importances.sort_values(by=\'Importance\', ascending=False, inplace=True)\n# Plotting each feature importance\nfeat_importances.plot(kind=\'bar\', figsize=(12,8))'

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Train Linear Regressor using the package
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Predictions
y_pred_linear = linear_model.predict(X_test)

# Evaluate performance
mse_linear_pkg = mean_squared_error(y_test, y_pred_linear)
r2_linear_pkg = r2_score(y_test, y_pred_linear)
print(f'Linear Regression - Mean Squared Error: {mse_linear_pkg}')
print(f'Linear Regression - R-squared: {r2_linear_pkg}')

Linear Regression - Mean Squared Error: 14974.133860641092
Linear Regression - R-squared: 0.5271138687719741


In [17]:
'''# Predictions
y_pred = linear_model.predict(X_test)

mse_linear_pkg_test = mean_squared_error(y_test, y_pred)
r2_linear_pkg_test = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse_linear_pkg_test}')
print(f'R-squared: {r2_linear_pkg_test }')'''

"# Predictions\ny_pred = linear_model.predict(X_test)\n\nmse_linear_pkg_test = mean_squared_error(y_test, y_pred)\nr2_linear_pkg_test = r2_score(y_test, y_pred)\nprint(f'Mean Squared Error: {mse_linear_pkg_test}')\nprint(f'R-squared: {r2_linear_pkg_test }')"

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn import set_config

# Creating the final pipeline with LinearRegression
final_pipeline = Pipeline([
    ('num_preprocess', numerical_pipeline),
    ('cat_preprocess', categorical_pipeline),
    ('model', LinearRegression())  # Using LinearRegression here
])

# Set the display configuration for pipeline visualization
set_config(display='diagram')

# Displaying the pipeline structure
final_pipeline

Pipeline(steps=[('num_preprocess',
                 Pipeline(steps=[('imputer', SimpleImputer()),
                                 ('scaler', MinMaxScaler())])),
                ('cat_preprocess',
                 Pipeline(steps=[('imputer',
                                  SimpleImputer(strategy='most_frequent')),
                                 ('target_encode',
                                  TargetEncoder(cols=[0, 1, 2]))])),
                ('model', LinearRegression())])

In [19]:
'''# Train the pipeline
final_pipeline.fit(X_train, y_train)

# Predictions using the pipeline
y_pred_pipeline = final_pipeline.predict(X_test)

# Evaluate performance
mse_pipeline = mean_squared_error(y_test, y_pred_pipeline)
r2_pipeline = r2_score(y_test, y_pred_pipeline)

print(f'Pipeline - Mean Squared Error: {mse_pipeline}')
print(f'Pipeline - R-squared: {r2_pipeline}')'''

"# Train the pipeline\nfinal_pipeline.fit(X_train, y_train)\n\n# Predictions using the pipeline\ny_pred_pipeline = final_pipeline.predict(X_test)\n\n# Evaluate performance\nmse_pipeline = mean_squared_error(y_test, y_pred_pipeline)\nr2_pipeline = r2_score(y_test, y_pred_pipeline)\n\nprint(f'Pipeline - Mean Squared Error: {mse_pipeline}')\nprint(f'Pipeline - R-squared: {r2_pipeline}')"

In [20]:
import numpy as np

class SimpleLinearRegression:
    def __init__(self):
        self.weights = None
        self.bias = None

    def fit(self, X, y, learning_rate=0.00001, n_iters=1000):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(n_iters):
            y_predicted = np.dot(X, self.weights) + self.bias

            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)

            self.weights -= learning_rate * dw
            self.bias -= learning_rate * db

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

scratch_model = SimpleLinearRegression()
scratch_model.fit(X_train.values, y_train.values)

y_pred_scratch = scratch_model.predict(X_test.values)

mse_scratch = mean_squared_error(y_test, y_pred_scratch)
r2_scratch = r2_score(y_test, y_pred_scratch)

print(f'Linear Regression from Scratch - Mean Squared Error: {mse_scratch}')
print(f'Linear Regression from Scratch - R-squared: {r2_scratch}')


Linear Regression from Scratch - Mean Squared Error: 22295.894005875594
Linear Regression from Scratch - R-squared: 0.2958912243718087


In [21]:
'''# Predictions
y_pred = model.predict(X_test)

mse_scratch_test = mean_squared_error(y_test, y_pred)
r2_scratch_test = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse_scratch_test}')
print(f'R-squared: {r2_scratch_test}')'''

"# Predictions\ny_pred = model.predict(X_test)\n\nmse_scratch_test = mean_squared_error(y_test, y_pred)\nr2_scratch_test = r2_score(y_test, y_pred)\nprint(f'Mean Squared Error: {mse_scratch_test}')\nprint(f'R-squared: {r2_scratch_test}')"

In [22]:
print("Comparing Linear Regression Implementations:")

print("\nScikit-Learn Linear Regression:")
print(f'Mean Squared Error: {mse_linear_pkg}')
print(f'R-squared: {r2_linear_pkg}')

'''print("\nLinear Regression using Pipeline:")
print(f'Mean Squared Error: {mse_pipeline}')
print(f'R-squared: {r2_pipeline}')'''

print("\nLinear Regression from Scratch:")
print(f'Mean Squared Error: {mse_scratch}')
print(f'R-squared: {r2_scratch}')


Comparing Linear Regression Implementations:

Scikit-Learn Linear Regression:
Mean Squared Error: 14974.133860641092
R-squared: 0.5271138687719741

Linear Regression from Scratch:
Mean Squared Error: 22295.894005875594
R-squared: 0.2958912243718087


In [23]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn import set_config

# Assuming numerical_pipeline and categorical_pipeline have already been defined

# Scikit-Learn Linear Regression Pipeline
pipeline_pkg = Pipeline([
    ('num_preprocess', numerical_pipeline),
    ('cat_preprocess', categorical_pipeline),
    ('model', LinearRegression())  # Scikit-Learn Linear Regression
])

# Custom Scratch Linear Regression Pipeline (Note: For visualization purposes)
# We'll create a dummy pipeline to show it in parallel
class CustomModelWrapper:
    def __init__(self, model):
        self.model = model

    def fit(self, X, y):
        self.model.fit(X, y)
        return self

    def predict(self, X):
        return self.model.predict(X)

pipeline_scratch = Pipeline([
    ('num_preprocess', numerical_pipeline),
    ('cat_preprocess', categorical_pipeline),
    ('model', CustomModelWrapper(SimpleLinearRegression()))  # Custom Scratch Linear Regression
])

# Set the display configuration for pipeline visualization
set_config(display='diagram')

# Visualize the pipelines
pipeline_pkg  # For Scikit-Learn Linear Regression
pipeline_scratch  # For Custom Scratch Linear Regression


Pipeline(steps=[('num_preprocess',
                 Pipeline(steps=[('imputer', SimpleImputer()),
                                 ('scaler', MinMaxScaler())])),
                ('cat_preprocess',
                 Pipeline(steps=[('imputer',
                                  SimpleImputer(strategy='most_frequent')),
                                 ('target_encode',
                                  TargetEncoder(cols=[0, 1, 2]))])),
                ('model',
                 <__main__.CustomModelWrapper object at 0x7e844a2ed8d0>)])